In [1]:
import sys, os, gc
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import random
from datetime import datetime
import json
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

In [2]:
def vgg6(input_shape=(128, 128, 3)):


    model = tf.keras.models.Sequential(name='VGG6')

    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape, name='conv1'))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', name='conv2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv3'))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv4'))
    model.add(tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_2'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))
    model.add(tf.keras.layers.Dropout(0.25))

    ## Fully connected layers
    model.add(tf.keras.layers.Flatten())

    # model.add(tf.keras.layers.Dense(256, activation='relu', name='fc_1'))
    # model.add(tf.keras.layers.Dropout(0.25))
    
    model.add(tf.keras.layers.Dense(256, activation='relu', name='fc_2'))
    # model.add(tf.keras.layers.Dropout(0.25))
    
    # model.add(tf.keras.layers.Dense(64, activation='relu', name='fc_3'))
    model.add(tf.keras.layers.Dense(32, activation='relu', name='fc_4'))

    model.add(tf.keras.layers.Dense(8, activation='relu', name='fc_5'))
    # output layer
    model.add(tf.keras.layers.Dense(1, name='fc_out1'))
    model.add(tf.keras.layers.Dense(1, name='fc_out2'))

    return model

def esn_shuffle(array, seed=0):
        np.random.seed(seed)
        np.random.shuffle(array)
        return array

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)


In [3]:
model = vgg6()
model.summary()

Model: "VGG6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 126, 126, 16)      448       
_________________________________________________________________
conv2 (Conv2D)               (None, 124, 124, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 62, 62, 16)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 60, 60, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 58, 58, 32)        9248      
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 58, 58, 32)        128    

In [4]:
def esn_shuffle(array, seed=None):
        if seed is not None:
            np.random.seed(seed)
        np.random.shuffle(array)
        return array

In [5]:
seq = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.1))),
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 3), per_channel=0.5),
    iaa.Add((-5, 5), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.5),
    iaa.Sometimes(0.1, iaa.Grayscale(alpha=(0.0, 1.0))),
    iaa.Sometimes(0.50, iaa.Grayscale(alpha=1.)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(0, 255))
    ], random_order=True)


seqGray = iaa.Sequential([
    iaa.GammaContrast(gamma=(0.97,1.03)),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Affine(
        rotate=(0, 359),
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, 
        mode=ia.ALL, cval=(255, 255))
    ], random_order=True)

In [6]:
%%time 

data = np.load('samples/RGB_128x128_test_000.npz')
images_RGB = data['images'].astype(np.uint8)
labels_RGB = data['labels'] 
pgcIDs_RGB = data['pgcIDs']
N = images_RGB.shape[0]
filter_RGB = np.chararray(N)
filter_RGB[:] = 'c'
data.close() 
del data


data = np.load('samples/g_128x128_test_000.npz')
images_g = data['images'].astype(np.uint8)
labels_g = data['labels'] 
pgcIDs_g = data['pgcIDs']
N = images_g.shape[0]
filter_g = np.chararray(N)
filter_g[:] = 'g'
data.close() 
del data

data = np.load('samples/r_128x128_test_000.npz')
images_r = data['images'].astype(np.uint8)
labels_r = data['labels'] 
pgcIDs_r = data['pgcIDs']
N = images_r.shape[0]
filter_r = np.chararray(N)
filter_r[:] = 'r'
data.close() 
del data

data = np.load('samples/i_128x128_test_000.npz')
images_i = data['images'].astype(np.uint8)
labels_i = data['labels'] 
pgcIDs_i = data['pgcIDs']
N = images_i.shape[0]
filter_i = np.chararray(N)
filter_i[:] = 'i'
data.close() 
del data

CPU times: user 2.1 s, sys: 68.1 ms, total: 2.17 s
Wall time: 2.17 s


In [7]:
images_gri = np.concatenate((images_g, images_r, images_i))
labels_gri = np.concatenate((labels_g, labels_r, labels_i))
pgcIDs_gri = np.concatenate((pgcIDs_g, pgcIDs_r, pgcIDs_i))
filter_gri = np.concatenate((filter_g, filter_r, filter_i))

N_RGB = len(labels_RGB)
N_gri = len(labels_gri)

In [8]:
ia.seed(100)

indx = esn_shuffle(np.arange(N_gri), seed=200)
images_aug = seqGray(images=images_gri[indx][:N_RGB,:,:,:])
labels_aug = labels_gri[indx][:N_RGB]
pgcIDs_aug = pgcIDs_gri[indx][:N_RGB]
filter_aug = filter_gri[indx][:N_RGB]

n = len(images_aug)
p = int(n/2)
images_aug[:p] = 255 - images_aug[:p]

ia.seed(200)

images_aug = np.concatenate((seq(images=images_RGB),images_aug))
labels_aug = np.concatenate((labels_RGB,labels_aug))
pgcIDs_aug = np.concatenate((pgcIDs_RGB,pgcIDs_aug))
filter_aug = np.concatenate((filter_RGB,filter_aug))

indx = np.arange(len(images_aug))
indx = esn_shuffle(indx, seed=100)
images_test_aug = images_aug[indx]
labels_test_aug = labels_aug[indx]
pgcIDs_test_aug = pgcIDs_aug[indx]
filter_test_aug = filter_aug[indx]

In [9]:
batches = esn_shuffle(np.arange(50), seed=50)

for i in range(1,60):
    batches = np.concatenate((batches, esn_shuffle(np.arange(50), seed=i*50))) 

batches, len(batches)

(array([34, 36,  1, ...,  2, 29,  7]), 3000)

In [10]:
zp_dir = 'Uset0_npz/'
ckpt_dir = 'U0_model01_ckpt/'

def trainer(iter, batches, suffix=None):

  if suffix is None:
    suffix = ''

  batchNo = batches[iter]
  if True: 

    vgg_model = vgg6()
    vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])

    if iter>0:

      vgg_model = vgg6()
      vgg_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(), metrics=['mse', 'mae'])
      vgg_model.load_weights(ckpt_dir+str(iter-1)+suffix+".ckpt")

      # if batchNo % 2 == 0 : 
      #   for layer in vgg_model.layers[:-1]:
      #     layer.trainable = False

      with open(ckpt_dir+'evalDict'+suffix+'.json') as json_file:
        evalDict = json.load(json_file)
    else:
      evalDict = {}


    print(iter, batchNo)


    batchFile = npzname = '128x128_train_aug_'+'%02d'%(batchNo+1)+'.npz'
    data = np.load(zp_dir + batchFile)
    images_train_aug = data['images'].astype(np.uint8)
    labels_train_aug = data['labels']
    pgcIDs_train_aug = data['pgcIDs']
    data.close() 
    del data



    n_epochs=1
    vgg_model.fit(images_train_aug, labels_train_aug, 
                                      epochs=n_epochs, batch_size=64, 
                                      validation_data=(images_test_aug, labels_test_aug),
                                      verbose=1, shuffle=True)
    evalDict[iter] = {} 
    for key in vgg_model.history.history:
      evalDict[iter][key] = vgg_model.history.history[key][0]
    evalDict[iter]["batchNo"] = batchNo
    evalDict[iter]["batchFile"] = batchFile
    
    with open(ckpt_dir+"evalDict"+suffix+".json", "w", encoding ='utf8') as outfile:
      json.dump(evalDict, outfile, allow_nan=True, cls=NpEncoder)
    
    vgg_model.save_weights(ckpt_dir+str(iter)+suffix+".ckpt")

    del vgg_model
    tf.keras.backend.clear_session()
    del images_train_aug 
    del labels_train_aug
    del pgcIDs_train_aug

    gc.collect()

    return evalDict

In [ ]:
with tf.device('/gpu:0'):
    for i in range(0,3000):
        trainer(i, batches)

0 34
282/282 [==============================] - 8s 30ms/step - loss: 353.5992 - mse: 353.5992 - mae: 13.4286 - val_loss: 1164.4397 - val_mse: 1164.4397 - val_mae: 31.1877
1 36
282/282 [==============================] - 6s 21ms/step - loss: 131.4763 - mse: 131.4763 - mae: 9.1667 - val_loss: 1740.3619 - val_mse: 1740.3619 - val_mae: 35.2461
2 1
282/282 [==============================] - 6s 20ms/step - loss: 121.1458 - mse: 121.1458 - mae: 8.7629 - val_loss: 1505.1757 - val_mse: 1505.1757 - val_mae: 32.6068
3 38
282/282 [==============================] - 6s 20ms/step - loss: 105.1666 - mse: 105.1666 - mae: 8.2029 - val_loss: 1087.6438 - val_mse: 1087.6438 - val_mae: 25.7766
4 8
282/282 [==============================] - 6s 20ms/step - loss: 102.9671 - mse: 102.9671 - mae: 8.0609 - val_loss: 1006.0658 - val_mse: 1006.0658 - val_mae: 25.4412
5 16
282/282 [==============================] - 6s 20ms/step - loss: 92.4405 - mse: 92.4405 - mae: 7.6178 - val_loss: 813.5428 - val_mse: 813.5428 - va

50 34
282/282 [==============================] - 6s 20ms/step - loss: 36.0363 - mse: 36.0363 - mae: 4.5590 - val_loss: 29.3566 - val_mse: 29.3566 - val_mae: 4.0593
51 36
282/282 [==============================] - 6s 20ms/step - loss: 35.0923 - mse: 35.0923 - mae: 4.4874 - val_loss: 33.9786 - val_mse: 33.9786 - val_mae: 4.3241
52 1
282/282 [==============================] - 6s 20ms/step - loss: 35.5014 - mse: 35.5014 - mae: 4.5178 - val_loss: 27.8121 - val_mse: 27.8121 - val_mae: 3.9198
53 38
282/282 [==============================] - 6s 20ms/step - loss: 34.5608 - mse: 34.5608 - mae: 4.4551 - val_loss: 30.9950 - val_mse: 30.9950 - val_mae: 4.2279
54 8
282/282 [==============================] - 6s 20ms/step - loss: 34.3358 - mse: 34.3358 - mae: 4.4421 - val_loss: 26.8340 - val_mse: 26.8340 - val_mae: 3.8239
55 16
282/282 [==============================] - 6s 20ms/step - loss: 33.9309 - mse: 33.9309 - mae: 4.4141 - val_loss: 47.5750 - val_mse: 47.5750 - val_mae: 5.1187
56 35
282/282 [===

282/282 [==============================] - 6s 20ms/step - loss: 24.9855 - mse: 24.9855 - mae: 3.7758 - val_loss: 22.4567 - val_mse: 22.4567 - val_mae: 3.5012
101 36
282/282 [==============================] - 6s 20ms/step - loss: 26.9283 - mse: 26.9283 - mae: 3.8610 - val_loss: 40.1176 - val_mse: 40.1176 - val_mae: 4.9055
102 37
282/282 [==============================] - 6s 20ms/step - loss: 26.5671 - mse: 26.5671 - mae: 3.8761 - val_loss: 23.1512 - val_mse: 23.1512 - val_mae: 3.5747
103 28
282/282 [==============================] - 6s 20ms/step - loss: 25.1880 - mse: 25.1880 - mae: 3.7623 - val_loss: 24.1207 - val_mse: 24.1207 - val_mae: 3.5780
104 43
282/282 [==============================] - 6s 20ms/step - loss: 25.9189 - mse: 25.9189 - mae: 3.7962 - val_loss: 29.8074 - val_mse: 29.8074 - val_mae: 4.1102
105 49
282/282 [==============================] - 6s 20ms/step - loss: 26.1006 - mse: 26.1006 - mae: 3.8176 - val_loss: 23.7462 - val_mse: 23.7462 - val_mae: 3.6016
106 5
282/282 [==

150 8
282/282 [==============================] - 6s 20ms/step - loss: 22.4840 - mse: 22.4840 - mae: 3.5171 - val_loss: 19.3286 - val_mse: 19.3286 - val_mae: 3.1194
151 31
282/282 [==============================] - 6s 20ms/step - loss: 22.6806 - mse: 22.6806 - mae: 3.5637 - val_loss: 23.7686 - val_mse: 23.7686 - val_mae: 3.5687
152 5
282/282 [==============================] - 6s 20ms/step - loss: 23.0019 - mse: 23.0019 - mae: 3.5606 - val_loss: 20.4773 - val_mse: 20.4773 - val_mae: 3.2631
153 2
282/282 [==============================] - 6s 20ms/step - loss: 22.8691 - mse: 22.8691 - mae: 3.5493 - val_loss: 20.5207 - val_mse: 20.5207 - val_mae: 3.3205
154 16
282/282 [==============================] - 6s 20ms/step - loss: 22.1374 - mse: 22.1374 - mae: 3.5101 - val_loss: 22.8752 - val_mse: 22.8752 - val_mae: 3.5361
155 47
282/282 [==============================] - 6s 20ms/step - loss: 22.4077 - mse: 22.4077 - mae: 3.5169 - val_loss: 21.2384 - val_mse: 21.2384 - val_mae: 3.3415
156 33
282/28

200 33
282/282 [==============================] - 6s 20ms/step - loss: 20.9478 - mse: 20.9478 - mae: 3.3858 - val_loss: 22.9867 - val_mse: 22.9867 - val_mae: 3.5546
201 8
282/282 [==============================] - 5s 19ms/step - loss: 20.8073 - mse: 20.8073 - mae: 3.3825 - val_loss: 17.7904 - val_mse: 17.7904 - val_mae: 3.0145
202 31
282/282 [==============================] - 6s 20ms/step - loss: 20.5054 - mse: 20.5054 - mae: 3.3685 - val_loss: 19.9869 - val_mse: 19.9869 - val_mae: 3.1978
203 28
282/282 [==============================] - 6s 20ms/step - loss: 19.5857 - mse: 19.5857 - mae: 3.3048 - val_loss: 21.5659 - val_mse: 21.5659 - val_mae: 3.4278
204 20
282/282 [==============================] - 6s 20ms/step - loss: 20.8063 - mse: 20.8063 - mae: 3.3688 - val_loss: 19.8887 - val_mse: 19.8887 - val_mae: 3.2148
205 49
282/282 [==============================] - 6s 20ms/step - loss: 20.3152 - mse: 20.3152 - mae: 3.3548 - val_loss: 22.1125 - val_mse: 22.1125 - val_mae: 3.4813
206 18
282/

...

282/282 [==============================] - 6s 20ms/step - loss: 11.1845 - mse: 11.1845 - mae: 2.5246 - val_loss: 14.7718 - val_mse: 14.7718 - val_mae: 2.7110
2275 7
282/282 [==============================] - 6s 20ms/step - loss: 11.2958 - mse: 11.2958 - mae: 2.5109 - val_loss: 15.2401 - val_mse: 15.2401 - val_mae: 2.7961
2276 2
282/282 [==============================] - 6s 20ms/step - loss: 11.2947 - mse: 11.2947 - mae: 2.5049 - val_loss: 14.5056 - val_mse: 14.5056 - val_mae: 2.6458
2277 48
282/282 [==============================] - 5s 19ms/step - loss: 10.9845 - mse: 10.9845 - mae: 2.4915 - val_loss: 15.8820 - val_mse: 15.8820 - val_mae: 2.8380
2278 26
282/282 [==============================] - 6s 20ms/step - loss: 10.7636 - mse: 10.7636 - mae: 2.4789 - val_loss: 15.2185 - val_mse: 15.2185 - val_mae: 2.7227
2279 1
282/282 [==============================] - 6s 20ms/step - loss: 11.1618 - mse: 11.1618 - mae: 2.5204 - val_loss: 14.4666 - val_mse: 14.4666 - val_mae: 2.6397
2280 5
282/282 

282/282 [==============================] - 6s 20ms/step - loss: 11.3983 - mse: 11.3983 - mae: 2.5153 - val_loss: 15.5017 - val_mse: 15.5017 - val_mae: 2.7118
2324 30
282/282 [==============================] - 6s 20ms/step - loss: 11.1902 - mse: 11.1902 - mae: 2.4935 - val_loss: 14.5430 - val_mse: 14.5430 - val_mae: 2.6359
2325 29
282/282 [==============================] - 6s 20ms/step - loss: 10.8218 - mse: 10.8218 - mae: 2.4814 - val_loss: 15.8585 - val_mse: 15.8585 - val_mae: 2.8206
2326 32
282/282 [==============================] - 6s 20ms/step - loss: 11.2467 - mse: 11.2467 - mae: 2.4972 - val_loss: 13.9810 - val_mse: 13.9810 - val_mae: 2.6113
2327 10
282/282 [==============================] - 6s 20ms/step - loss: 11.0009 - mse: 11.0009 - mae: 2.4926 - val_loss: 14.8574 - val_mse: 14.8574 - val_mae: 2.7028
2328 6
282/282 [==============================] - 6s 20ms/step - loss: 11.0712 - mse: 11.0712 - mae: 2.4954 - val_loss: 14.8542 - val_mse: 14.8542 - val_mae: 2.7100
2329 43
282/2

282/282 [==============================] - 6s 20ms/step - loss: 10.8743 - mse: 10.8743 - mae: 2.4705 - val_loss: 15.1182 - val_mse: 15.1182 - val_mae: 2.6781
2373 1
282/282 [==============================] - 6s 20ms/step - loss: 10.8931 - mse: 10.8931 - mae: 2.4872 - val_loss: 15.2334 - val_mse: 15.2334 - val_mae: 2.7332
2374 27
282/282 [==============================] - 6s 20ms/step - loss: 10.7297 - mse: 10.7297 - mae: 2.4656 - val_loss: 14.5954 - val_mse: 14.5954 - val_mae: 2.6646
2375 20
282/282 [==============================] - 6s 20ms/step - loss: 11.3038 - mse: 11.3038 - mae: 2.5088 - val_loss: 15.9415 - val_mse: 15.9415 - val_mae: 2.8455
2376 11
282/282 [==============================] - 6s 20ms/step - loss: 11.0227 - mse: 11.0227 - mae: 2.4828 - val_loss: 14.4761 - val_mse: 14.4761 - val_mae: 2.6460
2377 42
282/282 [==============================] - 5s 19ms/step - loss: 10.9975 - mse: 10.9975 - mae: 2.4871 - val_loss: 16.2871 - val_mse: 16.2871 - val_mae: 2.8698
2378 36
282/2

282/282 [==============================] - 6s 20ms/step - loss: 10.9548 - mse: 10.9548 - mae: 2.4786 - val_loss: 15.2400 - val_mse: 15.2400 - val_mae: 2.7496
2422 23
282/282 [==============================] - 6s 20ms/step - loss: 10.6017 - mse: 10.6017 - mae: 2.4489 - val_loss: 15.1244 - val_mse: 15.1244 - val_mae: 2.7006
2423 46
282/282 [==============================] - 6s 20ms/step - loss: 11.2312 - mse: 11.2312 - mae: 2.5153 - val_loss: 14.5743 - val_mse: 14.5743 - val_mae: 2.6657
2424 0
282/282 [==============================] - 6s 20ms/step - loss: 10.9905 - mse: 10.9905 - mae: 2.4899 - val_loss: 16.2491 - val_mse: 16.2491 - val_mae: 2.9233
2425 20
282/282 [==============================] - 6s 20ms/step - loss: 10.8516 - mse: 10.8516 - mae: 2.4725 - val_loss: 14.3946 - val_mse: 14.3946 - val_mae: 2.6419
2426 30
282/282 [==============================] - 6s 20ms/step - loss: 10.8117 - mse: 10.8117 - mae: 2.4636 - val_loss: 14.2219 - val_mse: 14.2219 - val_mae: 2.6214
2427 16
282/2

282/282 [==============================] - 6s 20ms/step - loss: 10.6613 - mse: 10.6613 - mae: 2.4390 - val_loss: 15.6762 - val_mse: 15.6762 - val_mae: 2.7939
2471 16
282/282 [==============================] - 6s 20ms/step - loss: 10.7024 - mse: 10.7024 - mae: 2.4599 - val_loss: 15.1518 - val_mse: 15.1518 - val_mae: 2.7324
2472 5
282/282 [==============================] - 6s 20ms/step - loss: 10.7141 - mse: 10.7141 - mae: 2.4526 - val_loss: 14.7392 - val_mse: 14.7392 - val_mae: 2.6485
2473 31
282/282 [==============================] - 6s 20ms/step - loss: 10.7834 - mse: 10.7834 - mae: 2.4816 - val_loss: 16.5188 - val_mse: 16.5188 - val_mae: 2.8595
2474 41
282/282 [==============================] - 6s 20ms/step - loss: 10.6242 - mse: 10.6242 - mae: 2.4608 - val_loss: 15.4792 - val_mse: 15.4792 - val_mae: 2.7468
2475 23
282/282 [==============================] - 6s 20ms/step - loss: 10.5646 - mse: 10.5646 - mae: 2.4450 - val_loss: 14.8383 - val_mse: 14.8383 - val_mae: 2.6621
2476 46
282/2